install selenium
and
chrome driver (also define path)

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

install tesseract

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

restart runtime

In [ ]:
import os
os.kill(os.getpid(), 9)
#-----------OR-----------
# quit()
#-----------OR-----------
# exit()

import dependancies

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import cv2
from PIL import Image, ImageCms, ImageFilter
import pytesseract

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

helper functions

In [ ]:
def step1():
    # open webpage
    driver.get(URL)

    # save captcha
    imdata = driver.find_element(By.ID,"imgCaptcha")
    with open(path, 'wb') as file:
        file.write(imdata.screenshot_as_png)

def step2():
    # convert to inverted mask and save img_temp
    im = cv2.imread(path)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
    Mask = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,horizontal_kernel, iterations=2)
    #Mask = cv2.bitwise_not(Mask)
    cv2.imwrite("old.png", Mask)

    # open img_temp and reinvert mask
    img = Image.open("old.png")
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

    # paste mask on img and save new_temp_img
    background = Image.open(path)
    background = background.convert("RGBA")
    background.paste(img,mask=img)
    background.save("new.png","PNG")

def step3(im): #  solve captcha
    im = Image.open(im) # open last saved img
    im = im.crop((5,5,115,35)) # crop it
    # conver image to extractable form elements (deffer captcha styles)
    rgb = ImageCms.createProfile(colorSpace='sRGB')
    lab = ImageCms.createProfile(colorSpace='LAB')
    transform = ImageCms.buildTransform(inputProfile=rgb, outputProfile=lab, inMode='RGB', outMode='LAB')
    lab_im = ImageCms.applyTransform(im=im, transform=transform)
    l, a, b = lab_im.split()
    im=l # select an element which is most extractable
    im = im.filter(ImageFilter.MinFilter(3)) # filter it
    result = pytesseract.image_to_string(im) # send it to ocr and save results to a variable
    l=[]
    l.append(result.strip())
    if l[0]==" " or l[0]=="" : # if result will be empty then it will do above steps again untill it gets the result
        step1()
        step2()
        l[0]=step3("new.png")
    return l[0] # return final result (maybe right or wrong)

def step4(enroll,ans): # return data
    # site automation 
    enr = driver.find_element(By.ID,"txtEnrollNo") # get enrollment no. text box
    captex = driver.find_element(By.ID,"CodeNumberTextBox") # get captcha text box
    enr.send_keys(enroll) # send (type) given enrollment number to text box
    captex.send_keys(ans) # send (type) extracted captcha text to text box
    captex.send_keys(Keys.RETURN) # return (ENTER)

    ere = driver.find_element(By.ID,"lblMSG").text
    if ere == "ERROR: Incorrect captcha code, try again." : 
        return "err"
    if ere == "Your request count is reached to maximum limit, Please try again later." : 
        return "reqover"
    if ere == "No Data Found! Check Enrollment Number." : 
        return "nodata"

    name = driver.find_element(By.ID,"lblName").text
    ins = driver.find_element(By.ID,"lblInstName").text
    bra = driver.find_element(By.ID,"lblBranch").text
    l=driver.find_elements(By.XPATH, "//table[@id='grdv2']/tbody/tr[2]")
    l=tuple(l[0].text.split(" "))
    cpi,cgpa,tb = l[1],l[2],l[3]
    l=driver.find_elements(By.XPATH, "//table[@id='grdvLastExm']/tbody/tr[2]")
    l=tuple(l[0].text.split(" "))
    le = f"{l[0]} {l[1]} {l[2]} {l[3]} {l[4]} {l[5]} {l[6]}"
    dd,cb,spi = l[8],l[9],l[10]
    return [enroll,name,ins,bra,float(cpi),float(cgpa),int(tb),le,dd,int(cb),float(spi)]

def loop():
    # just a loop through different enrollment numbers
    global counter
    mynewlist = []
    for i in mylist :
        enroll = "{}".format(i)
        step1()
        step2()
        ans=step3("new.png")
        nr=step4(enroll,ans)
        if nr == "err" :
            mynewlist.append(enroll)
        elif nr == "reqover" :
            print("Change the SERVER!")
            break
        elif nr == "nodata" :
            df.loc[len(df)] = [enroll,"nodata","-","-","-","-","-","-","-","-","-"]
            counter += 1
            print(f"{counter}/{tc} {int(counter*100/tc)}%")
        else :
            df.loc[len(df)] = nr
            counter += 1
            print(f"{counter}/{tc} {int(counter*100/tc)}%")
    return mynewlist

main function

In [ ]:
try:
	# initiate webdriver and configure options
	chrome_options = webdriver.ChromeOptions()
	chrome_options.add_argument('--headless')
	chrome_options.add_argument('--no-sandbox')
	chrome_options.add_argument('--disable-dev-shm-usage')
	chrome_options.add_argument("--incognito")

	ser = Service("chromedriver")
	driver = webdriver.Chrome(service=ser,options=chrome_options)

	# define url and filename for download captcha_temp
	URL = "https://www.students.gtu.ac.in/"
	path="cap.jpg"
 
	# create empty dataframe for filling output data with same labels that input file has
	df = pd.read_json('{"Enrollment No.":{},"Name":{},"Institute":{},"Branch":{},"CPI":{},"CGPA":{},"Total Backlog":{},"Last Exam":{},"Declaration Date":{},"Current Backlog":{},"SPI":{}}')

	mylist = range(190280111001,190280111010+1) # give range of enrollment no. (here i given our batch's range)
	
	counter = 0
	tc = len(mylist)
 
	# main driver programm
	# loop runs untill all data has scraped if any server error not happens
	while 1:
		mynewlist=loop()
		if len(mynewlist) != 0:
			mylist = mynewlist
		else:
			break

	# save dataframe to excel file
	df.to_excel("out.xlsx")

finally:
	driver.close() # close the window
	driver.quit() # stop the driver
	# remove unnecessary files
	import os
	os.remove("cap.jpg")
	os.remove("old.png")
	os.remove("new.png")

👈 download <font color='yellow'>out.xlsx</font> from left side bar by double clicking it